In [1]:
import spacy
from ginza import *
import corpus
import os

In [2]:
nlp = spacy.load('ja_ginza')
type(nlp)

spacy.lang.ja.Japanese

In [27]:
doc = nlp('メロンパンにメロンは入ってないよ。メロンの香りはついているけど、そのものは入ってない。')
for sent in doc.sents:
    for token in sent:
        print(
            token.i,
            token.orth_,
            token.lemma_,
            token.norm_,
            token.morph.get("Reading"),
            token.pos_,
            token.morph.get("Inflection"),
            token.tag_,
            token.dep_,
            token.head.i,
            token.lemma
        )
    print('EOS')

0 メロンパン メロンパン メロンパン ['メロンパン'] PROPN [] 名詞-普通名詞-一般 obl 4 2828855326191692016
1 に に に ['ニ'] ADP [] 助詞-格助詞 case 0 9559806972112097637
2 メロン メロン メロン ['メロン'] NOUN [] 名詞-普通名詞-一般 nsubj 4 14794725600052603870
3 は は は ['ハ'] ADP [] 助詞-係助詞 case 2 16159406377086191121
4 入っ 入る 入る ['ハイッ'] VERB ['五段-ラ行;連用形-促音便'] 動詞-一般 ROOT 4 3825154392790238919
5 て てる てる ['テ'] AUX ['下一段-タ行;未然形-一般'] 助動詞 aux 4 4044592081468128983
6 ない ない ない ['ナイ'] AUX ['助動詞-ナイ;終止形-一般'] 助動詞 aux 4 1006397123117692312
7 よ よ よ ['ヨ'] PART [] 助詞-終助詞 mark 4 62145933630954701
8 。 。 。 ['。'] PUNCT [] 補助記号-句点 punct 4 13183560425307684641
EOS
9 メロン メロン メロン ['メロン'] NOUN [] 名詞-普通名詞-一般 nmod 11 14794725600052603870
10 の の の ['ノ'] ADP [] 助詞-格助詞 case 9 2917952641780334418
11 香り 香り 香り ['カオリ'] NOUN [] 名詞-普通名詞-一般 nsubj 13 17759557579444759237
12 は は は ['ハ'] ADP [] 助詞-係助詞 case 11 16159406377086191121
13 つい つく 付く ['ツイ'] VERB ['五段-カ行;連用形-イ音便'] 動詞-非自立可能 advcl 21 17593882164978163850
14 て て て ['テ'] SCONJ [] 助詞-接続助詞 mark 13 17929645466003512548
15 いる いる 居る ['イル'

In [3]:
def search_candidate(known_token, sent):
    result = set()
    for target_token in bunsetu_head_tokens(sent):
        if((target_token.head.i == known_token.i) or (target_token.i == known_token.head.i)):
            result.add(target_token.i)
    return sorted(list(result))

In [4]:
cp = corpus.Corpus(os.path.join('data','nucc','data001.txt'))

確認してください: "Ｘ：１：＊＊＊。"


In [5]:
dic = dict()

In [6]:
dic['喜ぶ'] = corpus.word(origin='喜ぶ')
dic['喜ぶ'].set_value(1)


1

In [17]:
cp.conversation[6]

{'speaker': 'F107',
 'content': 'そうしたらそのワンちゃんがなんかか喜んじゃって、で、あたしの方に走ってきて、とびついてきちゃってさ。別にあたしさあ、別にさっきなでただけなのにさあ、なんかすごーいなつかれちゃってね。'}

In [8]:
for line in cp.conversation[6:7]:
    doc = nlp(line['content'])
    for sent in doc.sents:
        for token in bunsetu_head_tokens(sent):
            if(token.lemma_ in dic.keys() and dic[token.lemma_].isvisited):
                print(token.lemma_)
                candidates = search_candidate(token, sent)
                for candidate_index in candidates:
                    candidate = sent[candidate_index]
                    if(candidate.lemma_ in dic.keys()):
                        dic[candidate.lemma_].add_score(corpus.extract(dic[token.lemma_], dic[candidate.lemma_]))
                    else:
                        newword = corpus.word(token=candidate, origin=candidate.lemma_)
                        newword.add_score(corpus.extract(dic[token.lemma_], newword))
                        dic[candidate.lemma_] = newword
            

喜ぶ


In [9]:
dic

{'喜ぶ': <corpus.word at 0x1159d0610>,
 'する': <corpus.word at 0x116513d60>,
 'ちゃん': <corpus.word at 0x116fda820>,
 'なん': <corpus.word at 0x116fcf3d0>,
 '走る': <corpus.word at 0x11707a370>}